In [ ]:
from fastai.conv_learner import *
from fastai.plots import *
from planet import fbeta_score
from fastai.models import *
PATH='data/xrays' #Data is stored in XRays folder (subset) or XRays-Full (full dataset)
!ls {PATH}
!ls {PATH}/models

In [ ]:
label_csv = f'{PATH}/two.csv' #two.csv or two_ns.csv
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)

In [ ]:
arch=dn121 #or resnet18, resnet34, inceptionv4, resnext101_64, dn121, wrn
sz=224 #Image size (224 or 299)
bs=32
tfms = tfms_from_model(arch, sz, max_zoom=1.05, aug_tfms=transforms_basic)

In [ ]:
data = ImageClassifierData.from_csv(PATH, 'images', label_csv, bs=bs, val_idxs=val_idxs, tfms=tfms, skip_header=True)
data.classes

In [ ]:
#data.trn_ds

In [ ]:
#Show one image (to ensure they've loaded properly)
#x,y = next(iter(data.val_dl))
#plt.imshow(data.val_ds.denorm(to_np(x))[1]);

In [ ]:
# Set up learner, Precompute activations
data = data.resize(sz, 'tmp') #(optional) resize images
learn = ConvLearner.pretrained(arch, data, precompute=True) #Load learner, pre-compute activations
print(data.classes)
weights = [60,1] # (optional) - assign class weights
class_weights = torch.FloatTensor(weights).cuda() # (optional)
learn.crit = nn.CrossEntropyLoss(weight=class_weights) # (optional)

In [ ]:
# Determine ideal learning rate
learn.freeze()
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
# Train fully-connected layer at end (rest is frozen)

lr = .01
learn.freeze()
learn.fit(lr, 5, cycle_len=1, cycle_mult=2)

In [ ]:
# perform test-time augmentation, generate confusion matrix
learn.freeze()
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
preds = np.argmax(probs, axis=1)
probs = probs[:,1]
print("Baseline accuracy: {}".format(y.mean()))
cm = confusion_matrix(y, preds)
plot_confusion_matrix(cm, data.classes)

In [ ]:
# Save model as-is
learn.save(f'two_cats_{sz}_wrn_partial_60_L17')

In [ ]:

# Train full model (unfreeze)
lr = 0.01
lrs = [lr/9, lr/3, lr]
learn.unfreeze()
learn.fit(lrs, 5, cycle_len=1, cycle_mult=2)

In [ ]:
# Generate another confusion matrix
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
preds = np.argmax(probs, axis=1)
probs = probs[:,1]
cm = confusion_matrix(y, preds)
with open('fff.txt', 'w+') as f:
    f.write(str(cm))
    
#print("Baseline accuracy: {}".format(y.mean()))
#print("test set accuracy: {}".format(accuracy_np(probs, y)))
plot_confusion_matrix(cm, data.classes)

In [ ]:
#Save final model
learn.save(f'two_cats_{sz}_unfrozen__dn121_60_sample_BB')